In [3]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import os
import pandas as pd
from matplotlib.animation import FuncAnimation
from matplotlib.animation import PillowWriter
# %matplotlib widget

In [6]:
def plot_temperature_profiles(directory, time_start, time_stop, gif = False):
    """
    Функция, которая строит графики температуры по глубине по нескольку сценариев деградации

    """
    layer_thickness = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.6, 0.8])
    scenarios = ['HD', 'MD', 'LD', 'ND']
    colors = ['#d8d8d8ff', '#11a0d7ff', "#029c63ff", '#7d4ebaff','#eb681fff', '#e61f3dff', "#7da0d3ff"]
    scenario_colors = {
        'ND': colors[0],
        'LD': colors[2],
        'MD': colors[3],
        'HD': colors[4]
    }

    # Подкрутим данные глубин, чтобы они отражали середину каждого слоя
    mid_layer_depths = np.cumsum(layer_thickness) - (layer_thickness / 2)
    # Переместим 0, чтобы он не был совсем на рамке
    adjusted_mid_layer_depths = np.abs(mid_layer_depths - mid_layer_depths[0])
    adjusted_mid_layer_depths[0] = 0.05
    time_range = pd.date_range(start=pd.to_datetime(time_start)- pd.DateOffset(hours=5), 
                               end=pd.to_datetime(time_stop)- pd.DateOffset(hours=5), freq='30min')
    
    if gif:
        fig, ax = plt.subplots(figsize=(7, 9))
        ax.set_xlabel('Средняя температура слоя, T (°C)', fontsize = 14)
        ax.set_ylabel('Глубина слоя, z (м)', fontsize = 14)
        ax.invert_yaxis()
        ax.grid(True)

        lines = {scenario: ax.plot([], [], marker='o', label=f'{scenario} Scenario', color=scenario_colors[scenario])[0] for scenario in scenarios}
        ax.legend(fontsize = 14)

        def init():
            for line in lines.values():
                line.set_data([], [])
            ax.set_xlim([min_temperature, max_temperature])
            ax.set_ylim([max_depth, min_depth])
            return lines.values()
        
        def update(frame):
            time = time_range[frame]
            for scenario, line in lines.items():
                file_path = next((f for f in os.listdir(directory) if scenario in f), None)
                if file_path:
                    file_path = os.path.join(directory, file_path)
                    with xr.open_dataset(file_path) as dataset:
                        temperatures = dataset['tsl'].sel(time=str(time)).values.squeeze() - 273.15
                        line.set_data(temperatures, adjusted_mid_layer_depths)
            ax.set_title(f'Температурные профили почвы в {time_range[frame] + pd.DateOffset(hours=5)}', fontsize = 14)
            ax.set_xlim([min_temperature, max_temperature])
            ax.set_ylim([max_depth, min_depth])
            return lines.values()

        all_temperatures = []
        for scenario in scenarios:
            file_path = next((f for f in os.listdir(directory) if scenario in f), None)

            if file_path:
                file_path = os.path.join(directory, file_path)
                with xr.open_dataset(file_path) as dataset:
                    # Select the data within the time range
                    temperatures = dataset['tsl'].sel(time=slice(str(time_range[0]), str(time_range[-1]))).values - 273.15
                    all_temperatures.extend(temperatures.flatten())
        all_temperatures = np.array(all_temperatures)
        min_temperature, max_temperature = np.nanmin(all_temperatures)-0.2, np.nanmax(all_temperatures)+0.2

        min_depth, max_depth = 0, adjusted_mid_layer_depths[-1] + 0.1

        ani = FuncAnimation(fig, update, frames=range(len(time_range)), init_func=init, blit=True)
        ani.save(f'gifs/temperature_profile_{time_range[0]+ pd.DateOffset(hours=5)}-{time_range[-1]+ pd.DateOffset(hours=5)}.gif', writer=PillowWriter(fps=2))
        plt.close(fig)
        # plt.show()

    else:
        # На случай если нам не нужны гифки
        for time in time_range:
            plt.figure(figsize=(6, 9))
            for scenario in scenarios:
                file_path = next((f for f in os.listdir(directory) if scenario in f), None)
                if not file_path:
                    continue 
                file_path = os.path.join(directory, file_path)
                with xr.open_dataset(file_path) as dataset:

                    temperatures = dataset['tsl'].sel(time=str(time)).values.squeeze() - 273.15
                    
                    plt.plot(temperatures, adjusted_mid_layer_depths, marker='o', label=f'{scenario} Scenario')
            
            plt.xlabel('Средняя температура слоя, T (°C)', fontsize = 12)
            plt.ylabel('Глубина слоя, z (м)', fontsize = 12)
            time_str = time.strftime('%Y-%m-%d %H:%M', fontsize = 12)
            plt.title(f'Температурные профили почвы в {time_str}', fontsize = 12)
            plt.grid(True)
            plt.legend(fontsize = 12)
            plt.gca().invert_yaxis()
            plt.show()


In [11]:
direc = 'experiment_1_dai/halfhourly'

plot_temperature_profiles(
    directory=direc,
    time_start='2019-11-11 00:00', 
    time_stop='2019-11-11 23:30',
    gif = True 
)